In [1]:
import h5py
import numpy as np
from skimage.metrics import peak_signal_noise_ratio, structural_similarity

with h5py.File('../kiwi_hyperspectral_4d_data_with_mask.h5', 'r') as f:
    # Create basic noise metrics on a few sample slices to establish baseline
    # Pick a few representative excitation wavelengths for testing
    excitation_wavelengths = [300, 350, 400, 450, 500]  # Example values, adjust to your actual data

    # Create dictionary to store original data samples for testing
    test_samples = {}

    for ex_wavelength in excitation_wavelengths:
        group_name = f'excitation_{ex_wavelength}'
        if group_name in f:
            # Get average cube for this excitation
            if 'average_cube' in f[group_name]:
                # Store a few emission bands for testing
                avg_cube = f[group_name]['average_cube'][:]
                # Select a few bands (beginning, middle, end)
                band_indices = [0, avg_cube.shape[0]//2, avg_cube.shape[0]-1]
                for band_idx in band_indices:
                    test_samples[f"ex{ex_wavelength}_band{band_idx}"] = avg_cube[band_idx]

    # Print some basic noise statistics
    for name, image in test_samples.items():
        print(f"Sample {name}:")
        print(f"  Min: {np.min(image):.4f}, Max: {np.max(image):.4f}")
        print(f"  Mean: {np.mean(image):.4f}, Std: {np.std(image):.4f}")
        print(f"  Signal-to-Noise Ratio: {np.mean(image)/np.std(image):.4f}")

Sample ex300_band0:
  Min: 11.2000, Max: 237.3000
  Mean: 40.6019, Std: 8.7898
  Signal-to-Noise Ratio: 4.6192
Sample ex300_band70:
  Min: 4.4000, Max: 78.9000
  Mean: 13.4735, Std: 2.9796
  Signal-to-Noise Ratio: 4.5219
Sample ex300_band140:
  Min: 1.6000, Max: 30.2000
  Mean: 5.7430, Std: 1.3336
  Signal-to-Noise Ratio: 4.3064
Sample ex350_band0:
  Min: 14.9000, Max: 180.4000
  Mean: 42.2566, Std: 8.9541
  Signal-to-Noise Ratio: 4.7192
Sample ex350_band70:
  Min: 5.0000, Max: 67.5000
  Mean: 14.0121, Std: 3.0434
  Signal-to-Noise Ratio: 4.6041
Sample ex350_band140:
  Min: 2.0000, Max: 29.4000
  Mean: 5.9886, Std: 1.3601
  Signal-to-Noise Ratio: 4.4032
Sample ex400_band0:
  Min: 17.0000, Max: 373.7000
  Mean: 45.7771, Std: 12.0766
  Signal-to-Noise Ratio: 3.7906
Sample ex400_band70:
  Min: 5.3000, Max: 79.2000
  Mean: 14.9543, Std: 3.4642
  Signal-to-Noise Ratio: 4.3168
Sample ex400_band140:
  Min: 2.0000, Max: 37.4000
  Mean: 6.3930, Std: 1.5202
  Signal-to-Noise Ratio: 4.2053
Sample

In [2]:
import matplotlib.pyplot as plt
from skimage.restoration import (denoise_bilateral, denoise_tv_chambolle,
                                denoise_nl_means, denoise_wavelet)
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
import bm3d  # Install via: pip install bm3d

# Function to visualize and compare results
# Fix the compare_denoising function to specify data_range
def compare_denoising(original, denoised_results, title="Denoising Comparison"):
    """
    Compare original image with denoised versions

    Parameters:
    -----------
    original : 2D array
        Original noisy image
    denoised_results : dict
        Dictionary with method names as keys and denoised images as values
    title : str
        Plot title
    """
    n_methods = len(denoised_results)
    fig, axes = plt.subplots(1, n_methods + 1, figsize=(4*(n_methods + 1), 4))

    # Plot original
    axes[0].imshow(original, cmap='viridis')
    axes[0].set_title("Original")
    axes[0].axis('off')

    # Calculate data range for metrics
    data_range = original.max() - original.min()

    # Plot denoised versions
    for i, (method_name, denoised) in enumerate(denoised_results.items()):
        axes[i+1].imshow(denoised, cmap='viridis')
        # Calculate PSNR and SSIM with explicit data_range
        p = psnr(original, denoised, data_range=data_range)
        s = ssim(original, denoised, data_range=data_range)
        axes[i+1].set_title(f"{method_name}\nPSNR: {p:.2f}, SSIM: {s:.4f}")
        axes[i+1].axis('off')

    plt.suptitle(title)
    plt.tight_layout()
    return fig
# Process each test sample with classical methods
results = {}
for name, original in test_samples.items():
    print(f"Processing {name}...")

    # Estimate noise level from image
    # For Poisson noise, the variance equals the mean, so sqrt(mean) is the standard deviation
    noise_sigma_est = np.sqrt(np.mean(original))
    print(f"  Estimated noise sigma: {noise_sigma_est:.4f}")

    # Initialize results dictionary for this sample
    denoised_images = {}

    # 1. Bilateral Filter - edge-preserving smoothing
    denoised_images["Bilateral"] = denoise_bilateral(original, sigma_color=0.1, sigma_spatial=1)

    # 2. Total Variation (TV) - edge-preserving smoothing with flat regions
    denoised_images["TV-Chambolle"] = denoise_tv_chambolle(original, weight=0.1)

    # 3. Non-Local Means - exploits self-similarity
    # Patch size and search window can be tuned
    denoised_images["NL-Means"] = denoise_nl_means(original, h=0.8*noise_sigma_est,
                                                  patch_size=5, patch_distance=6,
                                                  fast_mode=True)

    # 4. Wavelet Denoising - good for preserving edges
    denoised_images["Wavelet"] = denoise_wavelet(original, method='BayesShrink',
                                               mode='soft', wavelet='db4')

    # 5. BM3D - state-of-the-art patch-based denoising
    denoised_images["BM3D"] = bm3d.bm3d(original, sigma_psd=noise_sigma_est)

    # Plot and compare results
    fig = compare_denoising(original, denoised_images,
                           f"Denoising Results for {name}")

    # Store results for this sample
    results[name] = {
        'original': original,
        'denoised': denoised_images,
        'figure': fig
    }

    # Save figure
    fig.savefig(f"Denoising Standard With Mask/denoising_comparison_{name}.png", dpi=150)
    plt.close(fig)

Processing ex300_band0...
  Estimated noise sigma: 6.3720
Processing ex300_band70...
  Estimated noise sigma: 3.6706
Processing ex300_band140...
  Estimated noise sigma: 2.3964
Processing ex350_band0...
  Estimated noise sigma: 6.5005
Processing ex350_band70...
  Estimated noise sigma: 3.7433
Processing ex350_band140...
  Estimated noise sigma: 2.4472
Processing ex400_band0...
  Estimated noise sigma: 6.7659
Processing ex400_band70...
  Estimated noise sigma: 3.8671
Processing ex400_band140...
  Estimated noise sigma: 2.5284
Processing ex450_band0...
  Estimated noise sigma: 7.8293
Processing ex450_band70...
  Estimated noise sigma: 4.4800
Processing ex450_band140...
  Estimated noise sigma: 2.8813
Processing ex500_band0...
  Estimated noise sigma: 9.8832
Processing ex500_band70...
  Estimated noise sigma: 5.7363
Processing ex500_band140...
  Estimated noise sigma: 3.0552


In [4]:
import scipy.ndimage as ndi
from skimage.restoration import denoise_tv_bregman

# Anscombe transform to stabilize variance of Poisson noise
def anscombe_transform(image):
    """Apply Anscombe transform to convert Poisson noise to Gaussian noise"""
    return 2 * np.sqrt(np.maximum(image + 3/8, 0))

# Inverse Anscombe transform with bias correction
def inverse_anscombe_transform(image):
    """Apply inverse Anscombe transform with bias correction"""
    return (image/2)**2 - 3/8

# Function to apply Anscombe + denoiser + inverse Anscombe
def anscombe_denoise(image, denoiser, *args, **kwargs):
    """
    Apply Anscombe transform, denoise, and inverse Anscombe

    Parameters:
    -----------
    image : 2D array
        Input image with Poisson noise
    denoiser : function
        Denoising function to apply (e.g., bm3d.bm3d)
    *args, **kwargs :
        Arguments to pass to the denoiser

    Returns:
    --------
    denoised_image : 2D array
        Denoised image
    """
    # Apply Anscombe transform
    transformed = anscombe_transform(image)

    # Apply denoiser to transformed image
    denoised_transformed = denoiser(transformed, *args, **kwargs)

    # Apply inverse Anscombe transform
    denoised = inverse_anscombe_transform(denoised_transformed)

    # Ensure non-negative values
    denoised = np.maximum(denoised, 0)

    return denoised

# Process each test sample with Poisson-specific methods
poisson_results = {}
for name, original in test_samples.items():
    print(f"Processing {name} with Poisson-specific methods...")

    # Initialize results dictionary for this sample
    denoised_images = {}

    # 1. Anscombe + BM3D (very effective for Poisson noise)
    denoised_images["Anscombe+BM3D"] = anscombe_denoise(
        original, bm3d.bm3d, sigma_psd=1.0)  # Sigma=1.0 after Anscombe transform

    # 2. Anscombe + Wavelet
    denoised_images["Anscombe+Wavelet"] = anscombe_denoise(
        original, denoise_wavelet, method='BayesShrink', mode='soft', wavelet='db4')

    # 3. Total Variation with Poisson fidelity term
    # Note: This is an approximation; a true Poisson-TV would need specialized solvers
    denoised_images["TV-Bregman"] = denoise_tv_bregman(original, weight=0.1)

    # 4. Median filter (often effective for salt-and-pepper/shot noise)
    denoised_images["Median"] = ndi.median_filter(original, size=3)

    # Plot and compare results
    fig = compare_denoising(original, denoised_images,
                           f"Poisson Denoising Results for {name}")

    # Store results for this sample
    poisson_results[name] = {
        'original': original,
        'denoised': denoised_images,
        'figure': fig
    }

    # Save figure
    fig.savefig(f"Poisson Denoising/poisson_denoising_comparison_{name}.png", dpi=150)
    plt.close(fig)

Processing ex300_band0 with Poisson-specific methods...
Processing ex300_band70 with Poisson-specific methods...
Processing ex300_band140 with Poisson-specific methods...
Processing ex350_band0 with Poisson-specific methods...
Processing ex350_band70 with Poisson-specific methods...
Processing ex350_band140 with Poisson-specific methods...
Processing ex400_band0 with Poisson-specific methods...
Processing ex400_band70 with Poisson-specific methods...
Processing ex400_band140 with Poisson-specific methods...
Processing ex450_band0 with Poisson-specific methods...
Processing ex450_band70 with Poisson-specific methods...
Processing ex450_band140 with Poisson-specific methods...
Processing ex500_band0 with Poisson-specific methods...
Processing ex500_band70 with Poisson-specific methods...
Processing ex500_band140 with Poisson-specific methods...


In [13]:
# For 3D data (x, y, emission for a fixed excitation), BM4D is a powerful extension
# This requires installing the bm4d package: pip install bm4d
import bm4d

def process_with_bm4d(cube, sigma=None):
    """
    Process a 3D hyperspectral cube with BM4D

    Parameters:
    -----------
    cube : 3D array (bands, height, width)
        Hyperspectral cube to denoise
    sigma : float or None
        Noise standard deviation. If None, estimated from data.

    Returns:
    --------
    denoised_cube : 3D array
        Denoised hyperspectral cube
    """
    # Reshape to (height, width, bands) as expected by BM4D
    cube_reshaped = np.transpose(cube, (1, 2, 0))

    # Estimate noise if not provided
    if sigma is None:
        # For Poisson, estimate as sqrt(mean)
        sigma = np.sqrt(np.mean(cube_reshaped))

    # Apply BM4D
    denoised_reshaped = bm4d.bm4d(cube_reshaped, sigma)

    # Reshape back to original form
    denoised_cube = np.transpose(denoised_reshaped, (2, 0, 1))

    return denoised_cube

# Test BM4D on one excitation's cube
with h5py.File('../kiwi_hyperspectral_4d_data.h5', 'r') as f:
    for ex_wavelength in excitation_wavelengths:  # Just try the first one
        group_name = f'excitation_{ex_wavelength}'
        if group_name in f and 'average_cube' in f[group_name]:
            print(f"Processing full cube for excitation {ex_wavelength} with BM4D...")

            # Get the cube
            cube = f[group_name]['average_cube'][:]

            # Process with BM4D
            denoised_cube = process_with_bm4d(cube)

            # Compare a few bands
            for band_idx in [0, cube.shape[0]//2, cube.shape[0]-1]:
                # Plot original vs denoised for this band
                fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

                ax1.imshow(cube[band_idx], cmap='viridis')
                ax1.set_title(f"Original - Band {band_idx}")
                ax1.axis('off')

                ax2.imshow(denoised_cube[band_idx], cmap='viridis')
                ax2.set_title(f"BM4D Denoised - Band {band_idx}")
                ax2.axis('off')

                plt.tight_layout()
                plt.savefig(f"Advanced Denoising/bm4d_ex{ex_wavelength}_band{band_idx}.png", dpi=150)
                plt.close()

Processing full cube for excitation 300 with BM4D...
Processing full cube for excitation 350 with BM4D...
Processing full cube for excitation 400 with BM4D...
Processing full cube for excitation 450 with BM4D...
Processing full cube for excitation 500 with BM4D...


In [1]:
import numpy as np
import h5py
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
from tqdm import tqdm
import os

# Set up GPU memory growth to avoid OOM errors
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

# Create Noise2Void U-Net model
def build_n2v_model(input_shape):
    """Build a U-Net model suitable for Noise2Void training"""
    inputs = Input(input_shape)

    # Encoder
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)

    # Decoder
    up4 = Concatenate()([UpSampling2D(size=(2, 2))(conv3), conv2])
    conv4 = Conv2D(128, 3, activation='relu', padding='same')(up4)
    conv4 = Conv2D(128, 3, activation='relu', padding='same')(conv4)

    up5 = Concatenate()([UpSampling2D(size=(2, 2))(conv4), conv1])
    conv5 = Conv2D(64, 3, activation='relu', padding='same')(up5)
    conv5 = Conv2D(64, 3, activation='relu', padding='same')(conv5)

    outputs = Conv2D(1, 1, activation='linear')(conv5)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='mse')

    return model

# Create blind-spot mask for Noise2Void
def create_blind_spot_mask(batch_size, height, width, blind_spot_percentage=0.25):
    """Create a random blind spot mask for Noise2Void training"""
    # Create random mask (1 for blind spots, 0 for normal pixels)
    mask = np.zeros((batch_size, height, width, 1))

    # Randomly select percentage of pixels to be blind spots
    for i in range(batch_size):
        num_pixels = int(height * width * blind_spot_percentage)
        flat_idx = np.random.choice(height * width, num_pixels, replace=False)
        y_idx, x_idx = np.unravel_index(flat_idx, (height, width))
        mask[i, y_idx, x_idx, 0] = 1

    return mask.astype(np.bool)

# Noise2Void data generator
def n2v_data_generator(images, batch_size=8, blind_spot_percentage=0.25):
    """Generate training batches for Noise2Void"""
    num_images = len(images)
    height, width = images[0].shape

    while True:
        # Select random images for the batch
        indices = np.random.choice(num_images, batch_size, replace=(batch_size > num_images))
        batch_images = np.array([images[i] for i in indices])

        # Reshape for the model
        batch_x = batch_images.reshape(batch_size, height, width, 1)
        batch_y = np.copy(batch_x)

        # Create blind spots
        mask = create_blind_spot_mask(batch_size, height, width, blind_spot_percentage)

        # Replace blind spot pixels with neighborhood mean in input
        for i in range(batch_size):
            for y, x in zip(*np.where(mask[i, :, :, 0])):
                # Define neighborhood (excluding center pixel)
                y_min, y_max = max(0, y-1), min(height, y+2)
                x_min, x_max = max(0, x-1), min(width, x+2)

                # Get neighborhood values (excluding center)
                neighborhood = batch_x[i, y_min:y_max, x_min:x_max, 0].copy()
                if y_min <= y < y_max and x_min <= x < x_max:  # Check if center pixel is in neighborhood
                    # Create mask to exclude center pixel
                    center_y, center_x = y - y_min, x - x_min
                    mask_2d = np.ones_like(neighborhood, dtype=bool)
                    mask_2d[center_y, center_x] = False
                    neighborhood = neighborhood[mask_2d]

                # Replace with random neighbor or mean
                if len(neighborhood) > 0:
                    #batch_x[i, y, x, 0] = neighborhood[np.random.randint(len(neighborhood))]  # Random neighbor
                    batch_x[i, y, x, 0] = np.mean(neighborhood)  # Mean of neighbors

        yield batch_x, batch_y

# Function to extract training data from H5 file
def extract_training_data(h5_file, max_images_per_excitation=20):
    """Extract training images from hyperspectral data"""
    training_images = []

    with h5py.File(h5_file, 'r') as f:
        for group_name in f:
            if group_name.startswith('excitation_'):
                print(f"Processing {group_name} for training data...")

                # Get the average cube for this excitation
                if 'average_cube' in f[group_name]:
                    avg_cube = f[group_name]['average_cube'][:]

                    # Take a limited number of bands
                    num_bands = min(avg_cube.shape[0], max_images_per_excitation)
                    indices = np.linspace(0, avg_cube.shape[0]-1, num_bands, dtype=int)

                    for band_idx in indices:
                        # Normalize to 0-1 range for better training
                        band = avg_cube[band_idx]
                        if np.max(band) > np.min(band):
                            band = (band - np.min(band)) / (np.max(band) - np.min(band))
                        training_images.append(band)

                # Also use individual cubes if available
                for key in f[group_name]:
                    if key.startswith('cube_'):
                        cube = f[group_name][key]['data'][:]

                        # Take a limited number of bands
                        num_bands = min(cube.shape[0], max_images_per_excitation // 2)
                        indices = np.linspace(0, cube.shape[0]-1, num_bands, dtype=int)

                        for band_idx in indices:
                            band = cube[band_idx]
                            if np.max(band) > np.min(band):
                                band = (band - np.min(band)) / (np.max(band) - np.min(band))
                            training_images.append(band)

    print(f"Extracted {len(training_images)} training images")
    return training_images

# Train Noise2Void model
def train_noise2void(h5_file, output_model_path='n2v_model.h5', epochs=1, batch_size=8):
    """Train a Noise2Void model on hyperspectral data"""
    # Extract training images
    training_images = extract_training_data(h5_file)

    if not training_images:
        print("No training images found!")
        return None

    # Get image dimensions
    height, width = training_images[0].shape

    # Build model
    model = build_n2v_model((height, width, 1))
    print(model.summary())

    # Create data generator
    train_gen = n2v_data_generator(training_images, batch_size=batch_size)

    # Create directory for model if it doesn't exist
    os.makedirs(os.path.dirname(output_model_path) if os.path.dirname(output_model_path) else '.', exist_ok=True)

    # Train model
    steps_per_epoch = max(1, len(training_images) // batch_size)
    history = model.fit(
        train_gen,
        steps_per_epoch=steps_per_epoch,
        epochs=epochs,
        verbose=1
    )

    # Save the model
    model.save(output_model_path)
    print(f"Model saved to {output_model_path}")

    # Plot training loss
    plt.figure(figsize=(10, 4))
    plt.plot(history.history['loss'])
    plt.title('Noise2Void Training Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.savefig(output_model_path.replace('.h5', '_loss.png'))
    plt.close()

    return model

# Apply Noise2Void model to denoise images
def apply_noise2void(model, images):
    """Apply trained Noise2Void model to denoise images"""
    height, width = images[0].shape
    denoised_images = []

    for img in tqdm(images, desc="Denoising images"):
        # Normalize
        img_min, img_max = np.min(img), np.max(img)
        img_norm = (img - img_min) / (img_max - img_min) if img_max > img_min else img

        # Reshape for model
        img_input = img_norm.reshape(1, height, width, 1)

        # Get denoised image
        denoised = model.predict(img_input)[0, :, :, 0]

        # Rescale back to original range
        denoised = denoised * (img_max - img_min) + img_min

        denoised_images.append(denoised)

    return np.array(denoised_images)

# Process entire dataset with Noise2Void
def denoise_dataset_with_noise2void(input_file, output_file, model_path=None):
    """Denoise entire hyperspectral dataset with Noise2Void"""
    # Train or load model
    if model_path and os.path.exists(model_path):
        print(f"Loading model from {model_path}")
        model = tf.keras.models.load_model(model_path)
    else:
        print("Training new Noise2Void model...")
        model = train_noise2void(input_file, output_model_path=model_path if model_path else 'n2v_model.h5')

    if model is None:
        print("Failed to create or load model!")
        return

    # Open input and output files
    with h5py.File(input_file, 'r') as f_in, h5py.File(output_file, 'w') as f_out:
        # Copy metadata
        if 'metadata' in f_in:
            metadata_group = f_out.create_group('metadata')
            for key, value in f_in['metadata'].attrs.items():
                metadata_group.attrs[key] = value
            metadata_group.attrs['denoising_method'] = 'Noise2Void'
            metadata_group.attrs['denoising_date'] = str(np.datetime64('now'))

        # Process each excitation group
        for group_name in f_in:
            if group_name.startswith('excitation_'):
                print(f"Processing {group_name}...")

                # Create corresponding group in output
                group_out = f_out.create_group(group_name)

                # Copy wavelengths
                if 'wavelengths' in f_in[group_name]:
                    group_out.create_dataset('wavelengths', data=f_in[group_name]['wavelengths'][:])

                # Process average cube if present
                if 'average_cube' in f_in[group_name]:
                    avg_cube = f_in[group_name]['average_cube'][:]
                    print(f"  Denoising average cube of shape {avg_cube.shape}...")

                    # Apply denoising
                    denoised_cube = np.zeros_like(avg_cube)
                    for i in range(avg_cube.shape[0]):
                        band = avg_cube[i]
                        # Apply model to this band
                        denoised_band = apply_noise2void(model, [band])[0]
                        denoised_cube[i] = denoised_band

                    # Save denoised cube
                    group_out.create_dataset('average_cube', data=denoised_cube,
                                           compression='gzip', chunks=True)

                # Optionally process individual cubes (can be time-consuming)
                for dataset_name in f_in[group_name]:
                    if dataset_name.startswith('cube_'):
                        # Just copy individual cubes without denoising
                        f_in[group_name][dataset_name].copy(source=f_in[group_name][dataset_name],
                                                           dest=group_out)

                # Copy mask if present
                if 'mask' in f_in[group_name]:
                    f_in[group_name]['mask'].copy(source=f_in[group_name]['mask'],
                                                dest=group_out)

    print(f"Denoised dataset saved to {output_file}")

# Example usage
# denoise_dataset_with_noise2void('kiwi_hyperspectral_4d_data.h5',
#                               'kiwi_hyperspectral_4d_data_n2v_denoised.h5',
#                               model_path='n2v_model.h5')

In [2]:
denoise_dataset_with_noise2void('../kiwi_hyperspectral_4d_data.h5',
                              'kiwi_hyperspectral_4d_data_n2v_denoised.h5',
                              model_path='n2v_model.h5')

Training new Noise2Void model...
Processing excitation_300 for training data...
Processing excitation_310 for training data...
Processing excitation_320 for training data...
Processing excitation_330 for training data...
Processing excitation_340 for training data...
Processing excitation_350 for training data...
Processing excitation_360 for training data...
Processing excitation_370 for training data...
Processing excitation_380 for training data...
Processing excitation_390 for training data...
Processing excitation_400 for training data...
Processing excitation_410 for training data...
Processing excitation_420 for training data...
Processing excitation_430 for training data...
Processing excitation_440 for training data...
Processing excitation_450 for training data...
Processing excitation_460 for training data...
Processing excitation_470 for training data...
Processing excitation_480 for training data...
Processing excitation_490 for training data...
Processing excitation_500 f

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 512, 640,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 512, 640,  │        640 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 512, 640,  │     36,928 │ conv2d[0][0]      │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 256, 320,  │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 256, 320,  │     73,856 │ max_pooling2d[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 256, 320,  │    147,584 │ conv2d_2[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 128, 160,  │          0 │ conv2d_3[0][0]    │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 128, 160,  │    295,168 │ max_pooling2d_1[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 128, 160,  │    590,080 │ conv2d_4[0][0]    │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d       │ (None, 256, 320,  │          0 │ conv2d_5[0][0]    │
│ (UpSampling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 256, 320,  │          0 │ up_sampling2d[0]… │
│ (Concatenate)       │ 384)              │            │ conv2d_3[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 256, 320,  │    442,496 │ concatenate[0][0] │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 256, 320,  │    147,584 │ conv2d_6[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d_1     │ (None, 512, 640,  │          0 │ conv2d_7[0][0]    │
│ (UpSampling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 512, 640,  │          0 │ up_sampling2d_1[… │
│ (Concatenate)       │ 192)              │            │ conv2d_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_8 (Conv2D)   │ (None, 512, 640,  │    110,656 │ concatenate_1[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_9 (Conv2D)   │ (None, 512, 640,  │     36,928 │ conv2d_8[0][0]  

 Total params: 1,881,985 (7.18 MB)

 Trainable params: 1,881,985 (7.18 MB)

 Non-trainable params: 0 (0.00 B)

None
 46/315 ━━━━━━━━━━━━━━━━━━━━ 39:10 9s/step - loss: 0.0049

KeyboardInterrupt: 